In [1]:
from google.colab import drive
drive.mount('/content/drive')
video_path = "/content/drive/MyDrive/crow/crowd.mp4"

Mounted at /content/drive


In [2]:
import cv2
import os
import numpy as np


# Пути
video_path = "/content/drive/MyDrive/crow/crowd.mp4"
output_path = 'output_tracked.mp4'

# Открываем видео
cap = cv2.VideoCapture(video_path)

# Проверяем, удалось ли открыть видео
if not cap.isOpened():
    raise ValueError("Не удалось открыть видеофайл")

# Получаем параметры видео
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

print(f"Видео считывается. FPS: {fps}, Размер кадра: {width}x{height}")

# Настройка VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Проверяем, успешно ли создан VideoWriter
if not out.isOpened():
    raise ValueError("Не удалось создать VideoWriter")

Видео считывается. FPS: 29, Размер кадра: 1920x1080


In [3]:
# Установка YOLOv8
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [4]:
from ultralytics import YOLO

model = YOLO('yolov8l.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 83.7M/83.7M [00:00<00:00, 243MB/s]


In [5]:
# Цикл по кадрам
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Выполняем детекцию
    results = model(frame, conf=0.6, iou=0.45, verbose=False)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            cls = box.cls.item()
            conf = box.conf.item()

            if cls == 0:
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Рисуем прямоугольник
                cv2.rectangle(frame, (x1, y1), (x2, y2), (127, 255, 0), 2)

                # Подпись с уверенностью
                label = f"person {conf:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Записываем кадр в выходной файл
    out.write(frame)

# Освобождаем ресурсы
cap.release()
out.release()

print("✅ Обработка завершена.")

✅ Обработка завершена.


In [6]:
# Скачивание файла
from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>